Este notebook es para la sistematizacion sin necesariamente usar un diccionario, solo para extraer ideas/palabras principales

In [21]:
import spacy
import pandas as pd
import numpy as np
import string
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import textacy

import funciones as f

In [22]:
nlp = spacy.load('es_core_news_md')

In [23]:
df = pd.read_excel('info.xlsx', sheet_name='Hoja1')

test = df[['Sector','P1']].dropna()

In [24]:
def extract_nounphrases(x):
    doc = nlp(x)
    noun_phrases = list(textacy.extract.noun_chunks(doc, min_freq=1))
    return [phrase.text for phrase in noun_phrases]

In [25]:
test['res'] = test['P1'].apply(extract_nounphrases)
test

,Sector,P1,res
0,Academia,"Puedan existir más leyes a nivel nacional, gob...","[más leyes, nivel nacional, , gobernanza estru..."
1,Academia,"Aparte de leyes, compartir información el tema...","[Aparte de leyes, información, tema, cibersegu..."
2,Academia,"Tecnología, a nivel de persona paradigma de no...","[Tecnología, , a nivel, persona, paradigma, no..."
3,Academia,Resguardar los datos de toda la comunidad soci...,"[datos, la comunidad social, , ciberseguridad,..."
4,Cuerpo diplomático,"Uno de los temas a considerar, sería la creaci...","[Uno, temas, MARCO LEGAL, tema, ciberseguridad..."
5,Cuerpo diplomático,Una ley especifica de ciberseguridad es muy im...,"[ley, ciberseguridad, marco, ley, , de protecc..."
6,Cuerpo diplomático,Es importante fortalecer a los cuerpos de segu...,"[cuerpos, seguridad, condiciones, infraestruct..."
7,Cuerpo diplomático,"Vital poder aportar a la educación, como el po...","[educación, , como el poder, niños, informació..."
8,Sector Privado,Normar la protección de datos,"[protección, datos]"
9,Sector Privado,Alfabetización digital,[Alfabetización digital]


In [26]:
test_res = f.expand_ans(test, 'res')

c:\Users\shannon.cap\Documents\GitHub\sistematizacion_test\funciones.py:61: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  x[termino] = x[col].apply(lambda x: 1 if termino in x else 0)
c:\Users\shannon.cap\Documents\GitHub\sistematizacion_test\funciones.py:61: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  x[termino] = x[col].apply(lambda x: 1 if termino in x else 0)
c:\Users\shannon.cap\Documents\GitHub\sistematizacion_test\funciones.py:61: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling